### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [2]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [3]:
today = date.today()
tomorrow = today + timedelta(days=1)

day = today

In [4]:
data_var = day.strftime('%Y-%m-%d')
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos.to_csv(f'jogos_processados/{data_var}.csv', index=False)

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
7459,2023-11-19,Brasil Nbb,11:00,R10 Score Vasco da Gama,Brasília,1.26,3.75,148.5,1.88,1.88,-9.5,1.95,1.76,dQjkKSQi,0.793651,0.266667,0.531915,0.531915,0.060317,0.0,0.0,NaN,NaN,NaN,NaN,NaN,255.206,215.068217,0.842724,0.0,0.0,92.50,632.50,0.0,1.0,0.0,0.0,0.702873,0.000000,0.072426,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
7460,2023-11-19,Espanha Acb,08:30,Granada,MoraBanc Andorra,1.92,1.92,167.5,1.81,1.95,-1.5,2.06,1.75,lGNpDvmp,0.520833,0.520833,0.552486,0.512821,0.041667,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,551.00,180.20,1.0,0.0,0.0,0.0,0.000000,0.052657,0.115067,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
7461,2023-11-19,Espanha Acb,08:30,Manresa,Obradoiro CAB,1.48,2.75,169.5,1.82,1.99,-5.5,2.02,1.74,n7yJHKvT,0.675676,0.363636,0.549451,0.502513,0.039312,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,117.90,336.00,0.0,0.0,0.0,0.0,0.424598,0.063101,0.105314,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
7462,2023-11-19,Espanha Acb,08:30,Real Madrid,Unicaja,1.22,4.56,169.5,1.83,1.95,-9.5,2.01,1.77,IkSlCb2j,0.819672,0.219298,0.546448,0.512821,0.038970,0.0,0.0,NaN,NaN,109.546,11.212699,0.102356,126.192,60.565868,0.479950,0.0,0.0,117.39,118.90,0.0,0.0,0.0,0.0,0.817210,0.044896,0.089791,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
7463,2023-11-19,Espanha Acb,13:00,Baskonia,Valencia,1.58,2.46,162.5,1.81,1.95,-5.5,2.07,1.69,CAvE23iF,0.632911,0.406504,0.552486,0.512821,0.039415,0.0,0.0,NaN,NaN,202.176,87.856328,0.434554,173.280,77.610757,0.447892,0.0,0.0,259.44,289.80,0.0,0.0,0.0,0.0,0.308047,0.052657,0.142926,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7693,2023-11-19,Eua Ncaa,22:00,St. Louis,Wichita State,2.51,1.59,150.5,1.91,1.91,2.5,2.08,1.67,Eg14WlxP,0.398406,0.628931,0.523560,0.523560,0.027337,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,100.00,117.82,0.0,0.0,0.0,0.0,0.317336,0.000000,0.154621,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
7694,2023-11-19,Eua Ncaa,22:00,USC,Brown,1.06,11.50,144.5,1.88,1.79,-18.5,2.00,1.72,YsI0PQy1,0.943396,0.086957,0.531915,0.558659,0.030353,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,66.00,101.76,0.0,0.0,1.0,0.0,1.175509,0.034681,0.106446,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
7695,2023-11-19,Eua Ncaa,22:30,Houston,Dayton,1.13,6.60,130.5,1.83,1.83,-13.5,1.97,1.74,2ktEjhNt,0.884956,0.151515,0.546448,0.546448,0.036471,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.00,130.90,0.0,0.0,0.0,0.0,1.000744,0.000000,0.087674,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
7696,2023-11-19,Islândia Premier League Feminina,16:15,Njardvik F,Snaefell F,1.04,9.85,146.5,1.84,1.82,-21.5,1.95,1.72,I3mYP40c,0.961538,0.101523,0.543478,0.549451,0.063061,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,1.144097,0.007728,0.088629,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
69,14:00,Eua Ncaa,Southern Miss,Utah Valley,137.5,1.91,0.8681,Over
79,20:00,Eua Ncaa,Arizona,Texas Arlington,154.5,1.91,0.9186,Over
207,15:30,Eua Ncaa,Stephen F. Austin,Loyola Marymount,148.5,1.91,0.8446,Over
213,16:30,Eua Ncaa,Winthrop,Elon,153.5,1.91,0.8583,Over
